In [1]:
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline

In [2]:
df = pd.read_csv("consumer_complaints.csv")

/Users/adityaasthana/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


<h2> Machine Learning Model data creation </h2>
    <p> We will build a supervised leaning model.</p>
    <p> The model will predict if a given case will become customer disputed or not </p>
    

In [4]:
disputed = df.loc[df["consumer_disputed?"].isin(["Yes"])]
print("Total number of disputed customers: ", disputed.shape[0])
not_disputed = df.loc[df["consumer_disputed?"].isin(["No"])]
print("Total number of undisputed customers: ", not_disputed.shape[0])

Total number of disputed customers:  112134
Total number of undisputed customers:  443823


<h3> Features: product, submitted_via,state, company_response_to_consumer, timely_response, issue </h3>

In [5]:
products = df['product'].unique()
print(products)
print(len(products))

['Mortgage' 'Credit reporting' 'Student loan' 'Debt collection'
 'Credit card' 'Bank account or service' 'Consumer Loan' 'Money transfers'
 'Payday loan' 'Prepaid card' 'Other financial service']
11


In [6]:
submitted_via = df['submitted_via'].unique()
print(submitted_via)
print(len(submitted_via))

['Referral' 'Postal mail' 'Email' 'Web' 'Phone' 'Fax']
6


In [7]:
states = df['state'].unique()
print(states)
print(len(states))

['CA' 'NY' 'MD' 'GA' 'AZ' 'IL' 'NC' 'TX' 'DC' 'KY' 'RI' 'TN' 'AR' 'AL'
 'NJ' 'VA' 'FL' 'MN' 'AK' 'OH' 'OR' 'MO' 'LA' 'SC' 'OK' 'WA' 'PA' 'MI'
 'CO' 'KS' 'MA' 'NH' 'NV' 'WV' 'PR' 'DE' 'IN' 'UT' 'ME' 'NE' 'NM' 'WY'
 'CT' 'HI' 'ID' nan 'MS' 'WI' 'IA' 'MT' 'MH' 'VT' 'AE' 'SD' 'FM' 'VI' 'ND'
 'GU' 'MP' 'AP' 'AS' 'PW' 'AA']
63


In [8]:
company_response_to_consumer = df['company_response_to_consumer'].unique()
print(company_response_to_consumer)
print(len(company_response_to_consumer))

['Closed with explanation' 'Closed with monetary relief'
 'Closed with non-monetary relief' 'Closed' 'Untimely response'
 'In progress' 'Closed without relief' 'Closed with relief']
8


In [9]:
timely_response = df['timely_response'].unique()
print(timely_response)
print(len(timely_response))

['Yes' 'No']
2


<h3> Clean and tokenize text </h3>

In [10]:
#text analysis
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adityaasthana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adityaasthana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
#cleaning up the writing in issues
corpus = []
errors = 0
for i,row in df.iterrows():
    text = ''
    try:   
        #remove all characters other than alphabets
        text = re.sub('[^a-zA-Z]', ' ', row['issue'])
        #Convert to lowercase    
        text = text.lower()
        #remove tags, if any (although crudely done for now)    
        text=re.sub("</?.*?>"," <> ",text)
        # remove special characters and digits    
        text=re.sub("(\\d\\W)+"," ",text)
        ##Convert to list from string    
        text = text.split()    
        ##Stemming    
        ps=PorterStemmer()
        #Lemmatisation    
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in stop_words] 
        text = " ".join(text)
        #print(text)
        corpus.append(text)
    except TypeError as err:
        #print(err)
        errors += 1
        corpus.append(text)
        continue
print("Errors:{}".format(errors))
assert len(corpus) == len(df.index)

Errors:0


In [13]:
df['raw_text'] = corpus
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id,raw_text
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074,loan modification collection foreclosure
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080,loan servicing payment escrow account
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473,incorrect information credit report
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326,repaying loan
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067,false statement representation


In [14]:
# Use data only for valid states
data = df.loc[df["state"].notnull()]
data.shape


(551070, 19)

In [15]:
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id,raw_text
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074,loan modification collection foreclosure
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080,loan servicing payment escrow account
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473,incorrect information credit report
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326,repaying loan
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067,false statement representation


In [16]:
def getOutPut(x):
    if x=='Yes':
        value = 0
    else: 
        value = 1
    return value

In [17]:
#Convert timely_response to numerical 
data['t_r'] = data['timely_response'].apply(getOutPut)

<ipython-input-17-9ea824030085>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['t_r'] = data['timely_response'].apply(getOutPut)


In [18]:
#Convert output to numerical 
data['y'] = df['consumer_disputed?'].apply(getOutPut)

<ipython-input-18-5dcbf97edb43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = df['consumer_disputed?'].apply(getOutPut)


<h3> New Feature delta_time  = 'date_received' - 'date_sent_to_company' (in days) </h3>

In [19]:
from datetime import datetime
def get_delta_time(x, y):
    start = datetime.strptime(x, '%m/%d/%Y')
    end = datetime.strptime(y, '%m/%d/%Y')
    return (end - start).days

In [20]:
#data['delta_time'] = data[['date_received', 'date_sent_to_company']].apply(get_delta_time)
data['delta_time'] = data.apply(lambda x: get_delta_time(x['date_received'], x['date_sent_to_company']),axis=1)

<ipython-input-20-2b494b765925>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['delta_time'] = data.apply(lambda x: get_delta_time(x['date_received'], x['date_sent_to_company']),axis=1)


In [21]:
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,...,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id,raw_text,t_r,y,delta_time
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,...,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074,loan modification collection foreclosure,0,0,4
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,...,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080,loan servicing payment escrow account,0,0,4
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,...,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473,incorrect information credit report,0,1,19
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,...,Email,08/30/2013,Closed with explanation,Yes,Yes,510326,repaying loan,0,0,0
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,...,Web,08/30/2013,Closed with explanation,Yes,Yes,511067,false statement representation,0,0,0


<h3> Convert Categorical data to One hot encoding </h3>
    <p> Data includes : product, state, company_response_to_consumer, submitted_via

In [22]:
df_prod = pd.get_dummies(data['product'], prefix='prod')

In [23]:
df_prod.head()

,prod_Bank account or service,prod_Consumer Loan,prod_Credit card,prod_Credit reporting,prod_Debt collection,prod_Money transfers,prod_Mortgage,prod_Other financial service,prod_Payday loan,prod_Prepaid card,prod_Student loan
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,0


In [24]:
df_state = pd.get_dummies(data['state'], prefix='st')
print(df_state.shape)
df_state.head()

(551070, 62)


,st_AA,st_AE,st_AK,st_AL,st_AP,st_AR,st_AS,st_AZ,st_CA,st_CO,...,st_TN,st_TX,st_UT,st_VA,st_VI,st_VT,st_WA,st_WI,st_WV,st_WY
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_crtc = pd.get_dummies(data['company_response_to_consumer'], prefix='crtc')
print(df_crtc.shape)
df_crtc.head()

(551070, 8)


,crtc_Closed,crtc_Closed with explanation,crtc_Closed with monetary relief,crtc_Closed with non-monetary relief,crtc_Closed with relief,crtc_Closed without relief,crtc_In progress,crtc_Untimely response
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0


In [26]:
df_sv = pd.get_dummies(data['submitted_via'], prefix='sv')
print(df_sv.shape)
df_sv.head()


(551070, 6)


,sv_Email,sv_Fax,sv_Phone,sv_Postal mail,sv_Referral,sv_Web
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,1,0,0,0,0,0
4,0,0,0,0,0,1


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score

In [28]:
documents = data['raw_text'].tolist()
print(len(documents))

551070


In [29]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(documents)

In [30]:
columns = vectorizer.get_feature_names()
print(len(columns))

158


In [31]:
x = tfidf.toarray()

In [32]:
df_words = pd.DataFrame(x, columns = columns)

In [33]:
print(df_words.shape)
df_words.head()

(551070, 158)


,account,acct,action,adding,advance,advertising,amt,application,applied,apply,...,underwriting,unexpected,unsolicited,use,using,vehicle,verification,withdrawal,workout,wrong
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.422548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3> Combine various data frames to generate the Data for machine learning model to work with </h3>

In [34]:
print(df_prod.shape)
print(df_state.shape)
df1 = pd.concat([df_prod,df_state], axis=1)
print(df1.shape)
#df1.head()

(551070, 11)
(551070, 62)
(551070, 73)


In [35]:
df2 = pd.concat([df1,df_crtc], axis=1)
print(df2.shape)
df2.head()

(551070, 81)


,prod_Bank account or service,prod_Consumer Loan,prod_Credit card,prod_Credit reporting,prod_Debt collection,prod_Money transfers,prod_Mortgage,prod_Other financial service,prod_Payday loan,prod_Prepaid card,...,st_WV,st_WY,crtc_Closed,crtc_Closed with explanation,crtc_Closed with monetary relief,crtc_Closed with non-monetary relief,crtc_Closed with relief,crtc_Closed without relief,crtc_In progress,crtc_Untimely response
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [36]:
df3 = pd.concat([df2,df_sv], axis=1)
print(df3.shape)
df3.head()

(551070, 87)


,prod_Bank account or service,prod_Consumer Loan,prod_Credit card,prod_Credit reporting,prod_Debt collection,prod_Money transfers,prod_Mortgage,prod_Other financial service,prod_Payday loan,prod_Prepaid card,...,crtc_Closed with relief,crtc_Closed without relief,crtc_In progress,crtc_Untimely response,sv_Email,sv_Fax,sv_Phone,sv_Postal mail,sv_Referral,sv_Web
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
#df4 = pd.concat([df3,df_words], axis=1)
#print(df4.shape)
#df4.head()

In [38]:
df_orig = data[['delta_time', 't_r', 'y']]

In [39]:
df_orig.shape

(551070, 3)

In [40]:
df_final = pd.concat([df3,df_orig], axis=1)
print(df_final.shape)
df_final.head()

(551070, 90)


,prod_Bank account or service,prod_Consumer Loan,prod_Credit card,prod_Credit reporting,prod_Debt collection,prod_Money transfers,prod_Mortgage,prod_Other financial service,prod_Payday loan,prod_Prepaid card,...,crtc_Untimely response,sv_Email,sv_Fax,sv_Phone,sv_Postal mail,sv_Referral,sv_Web,delta_time,t_r,y
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,4,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,4,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,19,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
#Save data for ML model
df_final.to_csv('data.csv', index=False)